Global setup


In [2]:
n=4
k=2
q=41
eta=2
R.<x> = ZZ[]

Bob's setup

In [3]:
Ax = matrix([[randrange(0,q)*x^3+randrange(0,q)*x^2+randrange(0,q)*x+randrange(0,q), randrange(0,q)*x^3+randrange(0,q)*x^2+randrange(0,q)*x+randrange(0,q)], 
            [randrange(0,q)*x^3+randrange(0,q)*x^2+randrange(0,q)*x+randrange(0,q), randrange(0,q)*x^3+randrange(0,q)*x^2+randrange(0,q)*x+randrange(0,q)]])
Ax  #public

[                 16*x + 23 7*x^3 + 15*x^2 + 14*x + 30]
[ 9*x^3 + 37*x^2 + 4*x + 20     x^3 + 4*x^2 + 4*x + 33]

In [4]:
def binomial_centered():
    a1=randrange(0,2)
    a2=randrange(0,2)
    b1=randrange(0,2)
    b2=randrange(0,2)
#    return a1+a2-b1-b2
    return a1-b1
    return 0

In [5]:
sx = matrix([[binomial_centered()*x^3+binomial_centered()*x^2+binomial_centered()*x+binomial_centered()], 
            [binomial_centered()*x^3+binomial_centered()*x^2+binomial_centered()*x+binomial_centered()]])
sx  #private

[x^2]
[x^2]

In [6]:
ex = matrix([[binomial_centered()*x^3+binomial_centered()*x^2+binomial_centered()*x+binomial_centered()], 
            [binomial_centered()*x^3+binomial_centered()*x^2+binomial_centered()*x+binomial_centered()]])
ex #private

[   x^2 + x + 1]
[-x^3 + x^2 + x]

In [7]:
tx = matrix(((Ax*sx + ex) % (x^n+1) % q))
tx  #public

[30*x^3 + 13*x^2 - 6*x - 14]
[      7*x^3 + 13*x^2 - 9*x]

**Matrices for lattice attack**


In [8]:
def negacirculant(v):
    Nshift = matrix([[0, 0, 0, -1],             #Negacyclic shift matrix
                     [1, 0, 0, 0],
                     [0, 1, 0, 0],
                     [0, 0, 1, 0]])
    return sum([v[i]*Nshift^i for i in range(0, len(v))])

In [9]:
Ax

[                 16*x + 23 7*x^3 + 15*x^2 + 14*x + 30]
[ 9*x^3 + 37*x^2 + 4*x + 20     x^3 + 4*x^2 + 4*x + 33]

In [10]:
A = block_matrix([[negacirculant(Ax[0,0].padded_list(4)), negacirculant(Ax[0,1].padded_list(4))], 
                  [negacirculant(Ax[1,0].padded_list(4)), negacirculant(Ax[1,1].padded_list(4))]])
A

[ 23   0   0 -16| 30  -7 -15 -14]
[ 16  23   0   0| 14  30  -7 -15]
[  0  16  23   0| 15  14  30  -7]
[  0   0  16  23|  7  15  14  30]
[---------------+---------------]
[ 20  -9 -37  -4| 33  -1  -4  -4]
[  4  20  -9 -37|  4  33  -1  -4]
[ 37   4  20  -9|  4   4  33  -1]
[  9  37   4  20|  1   4   4  33]

In [11]:
tx

[30*x^3 + 13*x^2 - 6*x - 14]
[      7*x^3 + 13*x^2 - 9*x]

In [12]:
t = block_matrix([[column_matrix(1, 4, tx[0,0].padded_list(4))],
                [column_matrix(1, 4, tx[1,0].padded_list(4))]])
t

[-14]
[ -6]
[ 13]
[ 30]
[---]
[  0]
[ -9]
[ 13]
[  7]

In [13]:
I8 = identity_matrix(8)
I8

[1 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0]
[0 0 0 1 0 0 0 0]
[0 0 0 0 1 0 0 0]
[0 0 0 0 0 1 0 0]
[0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 0 1]

**Eve's attack**

Eve knows $A$ so she can set up $M$:

In [14]:
M = block_matrix([[A, I8, -t, q*I8]])
M

[ 23   0   0 -16  30  -7 -15 -14|  1   0   0   0   0   0   0   0| 14| 41   0   0   0   0   0   0   0]
[ 16  23   0   0  14  30  -7 -15|  0   1   0   0   0   0   0   0|  6|  0  41   0   0   0   0   0   0]
[  0  16  23   0  15  14  30  -7|  0   0   1   0   0   0   0   0|-13|  0   0  41   0   0   0   0   0]
[  0   0  16  23   7  15  14  30|  0   0   0   1   0   0   0   0|-30|  0   0   0  41   0   0   0   0]
[ 20  -9 -37  -4  33  -1  -4  -4|  0   0   0   0   1   0   0   0|  0|  0   0   0   0  41   0   0   0]
[  4  20  -9 -37   4  33  -1  -4|  0   0   0   0   0   1   0   0|  9|  0   0   0   0   0  41   0   0]
[ 37   4  20  -9   4   4  33  -1|  0   0   0   0   0   0   1   0|-13|  0   0   0   0   0   0  41   0]
[  9  37   4  20   1   4   4  33|  0   0   0   0   0   0   0   1| -7|  0   0   0   0   0   0   0  41]

In [15]:
M.hermite_form()

[             1              0              0              0              0              0              0  3638967270092    26709699637   -64886021687   -34933752377    65737781820    -1278875781    45899875826     -195456370    30340350372 -1330117646263  1095097685117 -2660326889167 -1432283847457  2695249054620   -52433907021  1881894908866    -8013711170  1243954365252]
[             0              1              0              0              0              0              0  2687091708384    19723016752   -47913234147   -25795834199    48542192151     -944349386    33893455641     -144329188    22403967352  -982187481548   808643686832 -1964442600027 -1057629202159  1990229878191   -38718324826  1389631681281    -5917496708   918562661432]
[             0              0              1              0              0              0              0  2913757537964    21386724001   -51954887408   -27971805395    52636900277    -1024008646    36752490268     -156503873    24293822406 -1065

In [20]:
A, U = M.transpose().hermite_form(transformation = true)
A.str(), U.str()

('[1 0 0 0 0 0 0 0]\n[0 1 0 0 0 0 0 0]\n[0 0 1 0 0 0 0 0]\n[0 0 0 1 0 0 0 0]\n[0 0 0 0 1 0 0 0]\n[0 0 0 0 0 1 0 0]\n[0 0 0 0 0 0 1 0]\n[0 0 0 0 0 0 0 1]\n[0 0 0 0 0 0 0 0]\n[0 0 0 0 0 0 0 0]\n[0 0 0 0 0 0 0 0]\n[0 0 0 0 0 0 0 0]\n[0 0 0 0 0 0 0 0]\n[0 0 0 0 0 0 0 0]\n[0 0 0 0 0 0 0 0]\n[0 0 0 0 0 0 0 0]\n[0 0 0 0 0 0 0 0]\n[0 0 0 0 0 0 0 0]\n[0 0 0 0 0 0 0 0]\n[0 0 0 0 0 0 0 0]\n[0 0 0 0 0 0 0 0]\n[0 0 0 0 0 0 0 0]\n[0 0 0 0 0 0 0 0]\n[0 0 0 0 0 0 0 0]\n[0 0 0 0 0 0 0 0]',
 '[     -9778939273936186455411       1685874604961260263280    -114141504767252527394439      67602027648583343621927    -104368793995270447785895      55462680735077722693004     103016494367993339431032     -42405737795085197430718    5777417717070230606288592                            0                            0                            0                            0                            0                            0                            0                            0                           

In [55]:
M.right_kernel()

Free module of degree 25 and rank 17 over Integer Ring
Echelon basis matrix:
[  1   0   0   0   0   0   0   0   0  29   5  21  21   2   9   0  13  -5  -3   4   9  -1  -3   3   2]
[  0   1   0   0   0   0   0   0   0  18  25   0   9  21  37   4   0   0  -1  -1   0   0  -1  -1  -1]
[  0   0   1   0   0   0   0   0   0   0  18  25  37   9  21  37   0   0   0  -1  -1   0   0  -1  -1]
[  0   0   0   1   0   0   0   0   0  40   9  23   4  15  18  29   7  -2  -2   2   4   0  -1   2   0]
[  0   0   0   0   1   0   0   0   0  34   4  40   8  27  15  25  33 -12  -6  10  23  -1  -8  10   5]
[  0   0   0   0   0   1   0   0   0   8  13   0   1  24  23  20  21  -7  -4   6  15   0  -6   6   3]
[  0   0   0   0   0   0   1   0   0  24  22  24   4   6  19  24   4  -1  -1   0   2   0  -1   0   0]
[  0   0   0   0   0   0   0   1   0   9  20   0   4  36  14  15   1   0   0   0   0   0  -1   0  -1]
[  0   0   0   0   0   0   0   0   1  18   2  33   0  27   2  20  38 -13  -6  12  27   0  -9  12   6]
[  0 

In [58]:
M.right_kernel(basis='LLL')

Free module of degree 25 and rank 17 over Integer Ring
Echelon basis matrix:
[  1   0   0   0   0   0   0   0   0  29   5  21  21   2   9   0  13  -5  -3   4   9  -1  -3   3   2]
[  0   1   0   0   0   0   0   0   0  18  25   0   9  21  37   4   0   0  -1  -1   0   0  -1  -1  -1]
[  0   0   1   0   0   0   0   0   0   0  18  25  37   9  21  37   0   0   0  -1  -1   0   0  -1  -1]
[  0   0   0   1   0   0   0   0   0  40   9  23   4  15  18  29   7  -2  -2   2   4   0  -1   2   0]
[  0   0   0   0   1   0   0   0   0  34   4  40   8  27  15  25  33 -12  -6  10  23  -1  -8  10   5]
[  0   0   0   0   0   1   0   0   0   8  13   0   1  24  23  20  21  -7  -4   6  15   0  -6   6   3]
[  0   0   0   0   0   0   1   0   0  24  22  24   4   6  19  24   4  -1  -1   0   2   0  -1   0   0]
[  0   0   0   0   0   0   0   1   0   9  20   0   4  36  14  15   1   0   0   0   0   0  -1   0  -1]
[  0   0   0   0   0   0   0   0   1  18   2  33   0  27   2  20  38 -13  -6  12  27   0  -9  12   6]
[  0 

In [53]:
M.integer_kernel(ZZ)

Free module of degree 8 and rank 0 over Integer Ring
Echelon basis matrix:
[]

In [32]:
s = block_matrix([[column_matrix(1, 4, sx[0,0].padded_list(4))],
                [column_matrix(1, 4, sx[1,0].padded_list(4))]])
s

[0]
[0]
[1]
[0]
[-]
[0]
[0]
[1]
[0]

In [33]:
e = block_matrix([[column_matrix(1, 4, ex[0,0].padded_list(4))],
                [column_matrix(1, 4, ex[1,0].padded_list(4))]])
e

[ 1]
[ 1]
[ 1]
[ 0]
[--]
[ 0]
[ 1]
[ 1]
[-1]

In [43]:
matrix([1])

[1]

In [49]:
block_matrix([[s], [e], [matrix([1])], [matrix([[0],[0],[0],[0],[0],[0],[0],[0]])]])
_.str()

'[ 0]\n[ 0]\n[ 1]\n[ 0]\n[ 0]\n[ 0]\n[ 1]\n[ 0]\n[--]\n[ 1]\n[ 1]\n[ 1]\n[ 0]\n[ 0]\n[ 1]\n[ 1]\n[-1]\n[--]\n[ 1]\n[--]\n[ 0]\n[ 0]\n[ 0]\n[ 0]\n[ 0]\n[ 0]\n[ 0]\n[ 0]'

In [52]:
M*block_matrix([[s], [e], [matrix([1])], [matrix([[0],[0],[-1],[0],[1],[0],[-1],[0]])]])

[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]

In [365]:
B, Transform = M.LLL(transformation=true)
B

[ 0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0]
[ 0  0 -1  0  0  0  0  0]
[-1  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  1]
[ 0  0  0  0 -1  0  0  0]
[ 0  1  0  0  0  0  0  0]
[ 0  0  0 -1  0  0  0  0]
[ 0  0  0  0  0  1  0  0]
[ 0  0  0  0  0  0  1  0]

In [366]:
Transform

[   119148362543   -151049731431    185002633596   -455487849422    -73828177742    -78972991525    -65570051714    420624928190   -358921368005               0               0               0               0               0               0               0]
[  1100981003159  -1395763074620   1709502176763  -4208899381270   -682203426524   -729743670633   -605894867313   3886751320044  -3316584461335               5               0               0               0               0               0               0]
[  -232550731993    294815009303   -361083416887    889009555285    144095952384    154137468573    127977952844   -820964995721    700533562327               0               1               0               0               0               0               0]
[ 17040576666002 -21603104515339  26459042272532 -65143787567869 -10558892258935 -11294702570049  -9377816609334  60157699052828 -51332867343315              82               0               1               0               0  

In [367]:
Transform.determinant()

1

In [368]:
Transform*M == B

True

Solve $M\mathbf{x} = \mathbf{t}$

In [369]:
xvector = B.solve_left(t.transpose())
xvector

[  0   0   0   0   0   0   0   0 -24 -20  22   5  32  -8  20  26]

In [370]:
xhat = xvector.apply_map(round, R=ZZ)
xhat

[  0   0   0   0   0   0   0   0 -24 -20  22   5  32  -8  20  26]

In [371]:
xhat*Transform % q

[ 6  4  3 30  3 20 34 26 34 12  0  0  0  0  0  0]

In [372]:
that = xhat*B
that

[20 32 24  8 -5 20 26 22]

In [373]:
t.transpose()-that

[0 0 0 0 0 0 0 0]

Solve $B\hat{\mathbf{s}} = \hat{\mathbf{t}}$

In [374]:
shat = M.solve_left(that, check=true)
shat

[ 43170347092/71784273601  34963773604/71784273601 -16472552802/71784273601 -24386324333/71784273601 -15294437400/71784273601 -13099239250/71784273601  11816598749/71784273601  15188604885/71784273601                        0                        0                        0                        0                        0                        0                        0                        0]

In [375]:
shat % q

[ 6  4  3 30  3 20 34 26  0  0  0  0  0  0  0  0]

In [376]:
sx

[x^2 + 1]
[   -x^2]

In [379]:
s-shat.transpose()[0:8]

[ 28613926509/71784273601]
[ 36820499997/71784273601]
[ 88256826403/71784273601]
[ 24386324333/71784273601]
[-56489836201/71784273601]
[ 84883512851/71784273601]
[-11816598749/71784273601]
[ 56595668716/71784273601]

In [377]:
s = matrix([[1], [1], [1], [0],[-1], [1], [0], [1]])
s

[ 1]
[ 1]
[ 1]
[ 0]
[-1]
[ 1]
[ 0]
[ 1]

**Alternative:  embedding**  (Knospe, p. 280)

In [380]:
AZ = A.augment(q*I8)
AZ

[ 35 -24 -24 -25  36 -32  -8  -1  41   0   0   0   0   0   0   0]
[ 25  35 -24 -24   1  36 -32  -8   0  41   0   0   0   0   0   0]
[ 24  25  35 -24   8   1  36 -32   0   0  41   0   0   0   0   0]
[ 24  24  25  35  32   8   1  36   0   0   0  41   0   0   0   0]
[  2 -29 -26 -32   2 -17 -19 -40   0   0   0   0  41   0   0   0]
[ 32   2 -29 -26  40   2 -17 -19   0   0   0   0   0  41   0   0]
[ 26  32   2 -29  19  40   2 -17   0   0   0   0   0   0  41   0]
[ 29  26  32   2  17  19  40   2   0   0   0   0   0   0   0  41]

In [417]:
AZ.transpose().LLL().transpose()

[ 0  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0]
[ 0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0 -1  0  0]
[ 0  0  0  0  0  0  0  0  0  0  0 -1  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1]
[ 0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0]

In [381]:
H = A.transpose().hermite_form(include_zero_rows=false).transpose()
H

[           1            0            0            0            0            0            0            0]
[           0            1            0            0            0            0            0            0]
[           0            0            1            0            0            0            0            0]
[           0            0            0            1            0            0            0            0]
[           0            0            0            0            1            0            0            0]
[           0            0            0            0            0            1            0            0]
[           1            4            3            2            3            0            5            0]
[ 67740208903 192201946096   9159094559 110193022648 100076256650  17309173683 138029711320 358921368005]

In [413]:
B = ((H.augment(t))).stack(vector([0,0,0,0,0,0,0,0,1])) % q
B

[ 1  0  0  0  0  0  0  0 20]
[ 0  1  0  0  0  0  0  0 32]
[ 0  0  1  0  0  0  0  0 24]
[ 0  0  0  1  0  0  0  0  8]
[ 0  0  0  0  1  0  0  0 36]
[ 0  0  0  0  0  1  0  0 20]
[ 1  4  3  2  3  0  5  0 26]
[ 6 33  9 30 22 36 36 18 22]
[ 0  0  0  0  0  0  0  0  1]

In [414]:
B.transpose().LLL().transpose()

[ 0 -1  0  1 -1  0  0  0  0]
[ 0  1  0 -1 -1 -1  1  0  0]
[ 0 -1  1 -1  1 -1  0  1  0]
[ 0  0  0  0  1 -1 -1  1 -1]
[ 0  0  0  0  0  0 -1  1  1]
[-1  0  0  0  0  0  0  0  0]
[ 0  0  0 -1  0  1 -1  0 -1]
[ 0  0  1  0  0  0 -1 -1  0]
[ 0  0  1  0  0  0  0  1 -1]

In [409]:
e = block_matrix([[column_matrix(1, 4, ex[0,0].padded_list(4))],
                [column_matrix(1, 4, ex[1,0].padded_list(4))]])
e

[ 1]
[-1]
[ 1]
[ 1]
[--]
[ 0]
[ 0]
[ 0]
[ 1]

Still doesn't work.  Is the error vector too large?  (Is any reasonable error vector too large?)